In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-llroznim
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-llroznim


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/공모전/tmp/hotel_review_label1.csv')

In [ ]:
del data['Unnamed: 0']

In [ ]:
data.head()

,nickname,score,review,stay_date,label1
0,334sunmiy,50,노보텔 동대문 신규 오픈시점에 숙박하게됐습니다 침구도 푹신하고 침대마다 USB po...,2018-06-01,1.0
1,jooyeuns,30,이그제큐티브 룸 예약해서 라운지 이용했습니다 라운지 음식은 그렇게 좋은 퀄리티는 아...,2018-06-01,0.0
2,Young hoon L,50,지난 6월 30일 와이프의 강력한 요청으로 작년에 갔던 그랜드머큐어를 다시 가려고 ...,2018-06-01,1.0
3,Jiyoung W,50,아이파크몰이랑 가까워 먹거리 볼거리 즐길거리 다 있네요 스위트룸 컨디션도 좋았고 한...,2018-06-01,1.0
4,수지 김,40,평소에 호텔에 관심이 많아서 같은 도시 내에서도 많이 다니는 편인데 드래곤시티가 새...,2018-06-01,1.0


In [ ]:
data['label1'] =data['label1'].replace(11, 1)

In [ ]:
data.label1.value_counts()

1.0    8723
0.0     685
Name: label1, dtype: int64

In [ ]:
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [ ]:
# dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [ ]:
new_data = data[['review', 'label1']]

In [ ]:
new_data.head()

,review,label1
0,노보텔 동대문 신규 오픈시점에 숙박하게됐습니다 침구도 푹신하고 침대마다 USB po...,1.0
1,이그제큐티브 룸 예약해서 라운지 이용했습니다 라운지 음식은 그렇게 좋은 퀄리티는 아...,0.0
2,지난 6월 30일 와이프의 강력한 요청으로 작년에 갔던 그랜드머큐어를 다시 가려고 ...,1.0
3,아이파크몰이랑 가까워 먹거리 볼거리 즐길거리 다 있네요 스위트룸 컨디션도 좋았고 한...,1.0
4,평소에 호텔에 관심이 많아서 같은 도시 내에서도 많이 다니는 편인데 드래곤시티가 새...,1.0


In [ ]:
new_data.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42) # new_df 오탈자 수정

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
train

,review,label1
2128,젊은 나이였던 나에게 호텔이라는 곳은 신혼여행 가기전의 1박이나 아주아주 특별한 날...,1.0
1509,20일날이 여자친구 생일이었는데 여자친구가 여기 와보고 싶다해서 예약하게 되었습니다...,1.0
7207,아주 쾌적하고 좋아요친절해서 좋았어요 시설도 시원하고 잘챙겨주세요 교통 편리하고 커...,1.0
3651,공항에서 가까운 숙소를 찾다 라마다시티호텔 후기를 보고 이용하게 되었다 4인이 이용...,1.0
5515,두번째 방문입니다 가격대비 서비스 편안함으로는 그 어떤 호텔보다 최고의 서비스를 자...,1.0
...,...,...
5734,방 너무 깔끔하고 직원분들 역시 친절하시고 시티뷰인데 해운대도 꽤 많이 보여요 업그...,1.0
5191,30번째 제 생일기념으로 남자친구랑 방문했는데 전반적으로 모든게 만족스러웠던 호캉스...,1.0
5390,가족 휴식 겸 부모님 편히 묵으시라고 예약했는데 깔끔하고 좋아요 다만 다른 지점들...,0.0
860,위치 시설 방 너무 좋았어요 아래 편의점도 있고 주차 시설도 넓습니다 체크인도 빠르...,1.0


In [ ]:
data_train = BERTDataset(train.values, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test.values, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 4073, 1397, 6947, 1370, 6897, 5095, 7103, 1021, 7086, 3010,
        7931, 6925,  517, 5330, 5561, 7207, 7095,  529, 6280, 7098, 3128,
        6797, 7276, 4780, 1407, 6902, 5037, 6015, 7096, 1765, 1021, 7119,
        4252, 3166, 6828, 3222, 5847, 4257, 5712, 7095, 1397, 6079, 4086,
        6855, 5931, 6199, 1988, 4928, 7227, 5859, 5760, 1370, 2934, 6897,
        3367, 2644,  517, 6989, 5439, 1700, 5025, 4998,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.6621145606040955 train acc 0.609375

epoch 1 train acc 0.9155539139161463


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8896233974358975


...


epoch 3 test acc 0.95


epoch 4 batch id 1 loss 0.049882493913173676 train acc 0.984375

epoch 4 train acc 0.9811064339875112



epoch 4 test acc 0.9541666666666667


epoch 5 batch id 1 loss 0.01603894867002964 train acc 1.0

epoch 5 train acc 0.9895391949152542



epoch 5 test acc 0.9552083333333333


In [ ]:
unseen_test = pd.read_csv('/content/drive/MyDrive/공모전/tmp/test_set_final.csv')

In [ ]:
unseen_test.head()

,Unnamed: 0,nickname,score,review,stay_date
0,0,karolinernes,50,이그제큐티브 라운지 칵테일 뷔페는 생각보다 넘 별로였어요 메뉴가 얼마 없어서 ifc...,2021-07-01
1,1,mikyung782021,50,직원분들 모두들 너무 친전하셨고 말하지 않아도 무엇이 필요한지 먼저 오퍼해 주셨어요...,2021-07-01
2,2,shinobu2,20,가격대비 메리크가 없으며 이제는 가격을 어느정도 지불해야지만 앞의 조망을 즐길수 있...,2021-07-01
3,3,sharpgoodman,50,가족들과 즐거운 여름 휴가를 보낼 수 있었습니다 더운 여름이었지만 11시쯤 걸어다닌...,2021-07-01
4,4,sunkeem,50,위치 주차 시설 서비스 등 호텔이 제공해야 할 모든면이 편리하고 좋았다 자녀들과 함...,2021-07-01


In [ ]:
del unseen_test['Unnamed: 0']

In [ ]:
dataset2 = unseen_test

In [ ]:
unseen_test = unseen_test[['review']]

In [ ]:
unseen_test['label1'] = 9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)
test_output = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  test_output.append(out.tolist())

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [ ]:
dataset2['sentimental_score'] = test_output

In [ ]:
# dataset2.to_csv('/content/drive/MyDrive/공모전/tmp/sentimental_score_kobert.csv', encoding = 'cp949')

In [ ]:
import pickle

In [ ]:
with open('/content/drive/MyDrive/공모전/tmp/kobert.pkl', 'wb') as f:
    pickle.dump(model, f)